In [ ]:
%load_ext cython
import os
import cbclib as cbc
import numpy as np
import pandas as pd
import pyximport
from importlib import reload
import sys
import hdf5plugin
import h5py
import pygmo
import matplotlib.animation as animation
from tqdm.auto import tqdm
from scipy import ndimage
from scipy.optimize import minimize, differential_evolution
from scipy.interpolate import interpn

import plotly.graph_objects as go
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.widgets import Slider
from pickle import load, dump

pyximport.install(reload_support=True, build_in_temp=False,
                  build_dir='.pyxbld')

In [ ]:
if sys.modules.get('dev'): # Maybe sys.modules is better?
    dev = sys.modules.get('dev')
    dev = reload(dev)
else:
    import dev
print(dir(dev))

In [ ]:
setup = cbc.ScanSetup.import_ini('results/exp_geom_232_ref_2.ini')
table = cbc.CBCTable.import_hdf('results/scan_232_modeled_part.h5', 'data', setup)
table_det = cbc.CBCTable.import_hdf('results/scan_232_detected.h5', 'data', setup)
crop = cbc.Crop(roi=(1100, 3260, 1040, 3108))
basis = cbc.Basis.import_ini('results/scan_232_basis_ref_2.ini')
samples = cbc.ScanSamples.import_dataframe(pd.read_hdf('results/scan_232_samples_log_2.h5', 'data'))

In [ ]:
scan_num = 232
dir_path = '/asap3/petra3/gpfs/p11/2021/data/11010570/raw'
h5_dir = os.path.join(dir_path, f'scan_frames/Scan_{scan_num:d}')
h5_files = sorted([os.path.join(h5_dir, path) for path in os.listdir(h5_dir)
                   if path.endswith(('LambdaFar.nxs', '.h5'))])
data = cbc.CrystData(cbc.CXIStore(h5_files), transform=crop)

wf_data = cbc.CrystData(cbc.CXIStore(f'results/scan_250_whitefield.h5'),
                        transform=crop).load('whitefield')

In [ ]:
idxs = np.arange(0, 10)

data = data.clear().load(idxs=idxs)
data = data.update_mask(method='range-bad', vmax=10000000)
data = data.mask_pupil(setup, padding=60)
data = data.import_whitefield(wf_data.whitefield)
data = data.blur_pupil(setup, padding=80, blur=20)

In [ ]:
data = data.import_patterns(table.table)
data = data.update_background()

In [ ]:
det_obj = data.lsd_detector()
det_obj = det_obj.generate_patterns(vmin=0.9, vmax=5.0, size=(1, 3, 3))

In [ ]:
det_res = det_obj.detect(cutoff=90.0, filter_threshold=12.0, group_threshold=0.65, dilation=1.5)

In [ ]:
det_res = det_res.update_patterns((1.5, 2.5, 8.5))

In [ ]:
det_res.patterns[0].min(), det_res.patterns[0].max()

In [ ]:
%matplotlib widget
fig, ax = plt.subplots(figsize=(8, 8))
ax.imshow(det_res.patterns[0], vmin=0.0, vmax=0.5, cmap='gray_r')
# for line in det_res.streaks[0].to_numpy():
#     ax.plot(line[:4:2], line[1:4:2], c='r', alpha=0.1)
ax.set_xlim(0, det_obj.shape[2])
ax.set_ylim(det_obj.shape[1], 0)
plt.show()

In [ ]:
mdl_det = data.model_detector(basis, samples, setup)

In [ ]:
mdl_det = mdl_det.detect(threshold=1.0, width=5.0)

In [ ]:
mdl_det = mdl_det.update_patterns((1.5, 2.5, 7.0))
table = mdl_det.export_table()

In [ ]:
model = mdl_det.models[0]

In [ ]:
width = 3.0

hkl = model.generate_hkl(0.3)
kin, mask = cbc.bin.calc_source_lines(basis=model.basis.mat, hkl=hkl, kin_min=model.setup.kin_min,
                                      kin_max=model.setup.kin_max)

foc_dist = (model.setup.foc_pos - model.sample.position)[2]
sx0, sy0 = model.setup._k_to_det(model.setup.kin_center,
                                 np.array([0.0, 0.0, foc_dist]))
sx, sy = model.setup._k_to_det(kin,
                               np.array([0.0, 0.0, foc_dist]))

hkl = hkl[mask]
kout = kin + hkl.dot(model.basis.mat)[:, None]

x, y = model.sample.kout_to_detector(kout, model.setup)
x, y = x + sx - sx0, y + sy - sy0
if model.transform:
    x, y = model.transform.forward_points(x, y)

if model.shape:
    mask = (0 < y).any(axis=1) & (y < model.shape[0]).any(axis=1) & \
           (0 < x).any(axis=1) & (x < model.shape[1]).any(axis=1)
    x, y, hkl = x[mask], y[mask], hkl[mask]
streaks = cbc.Streaks(x0=x[:, 0], y0=y[:, 0], x1=x[:, 1], y1=y[:, 1],
                      h=hkl[:, 0], k=hkl[:, 1], l=hkl[:, 2],
                      width=width * np.ones(x.shape[0]))

In [ ]:
%matplotlib widget
fig, ax = plt.subplots(figsize=(8, 8))
ax.imshow(mdl_det.data[0], vmin=0.5, vmax=5.0, cmap='gray_r')
for line in streaks.to_numpy():
    ax.plot(line[:4:2], line[1:4:2], alpha=0.3, c='r')
    
ax.set_ylim(mdl_det.shape[1], 0)
ax.set_xlim(0, mdl_det.shape[2])
plt.show()

In [ ]:
frame_idx = 63

table_det.setup = scan_setup
problem = table_det.refine_sample(frame_idx, (4e-2, 4e-3, 0.0), basis, samples[frame_idx],
                                  0.3, 3.0, 0.5)

In [ ]:
uda = pygmo.de(gen=300)
algo = pygmo.algorithm(uda)
prob = pygmo.problem(problem)
pops = [pygmo.population(size=10, prob=prob, b=pygmo.bfe()) for _ in range(32)]
archi = pygmo.archipelago()
for pop in pops:
    archi.push_back(algo=algo, pop=pop)

In [ ]:
archi.evolve()
%time archi.wait()
x = archi.get_champions_x()[np.argmin(archi.get_champions_f())]

In [ ]:
problem.fitness(problem.x0), problem.fitness(x)

In [ ]:
%matplotlib widget
idx = 0

fig, ax = plt.subplots(1, 1, figsize=(8, 8))
# ax.imshow(table.pattern_image(frame_idx), vmin=0.0, vmax=0.5, cmap='gray_r')
ax.imshow(divisor[idx], vmin=0, vmax=20)
# ax.imshow(data.data[0], cmap='gray_r', vmax=5.0)
# for line in mdl_det.streaks[idx].to_numpy():
#     ax.plot(line[:4:2], line[1:4:2], c='r', alpha=0.4)
# for line in problem.generate_streaks(problem.x0).to_numpy():
#     ax.plot(line[:4:2], line[1:4:2], c='r', alpha=0.4)
# for line in problem2.generate_streaks(x2).to_numpy():
#     ax.plot(line[:4:2], line[1:4:2], c='b', alpha=0.4)

# ax.set_xlim(0, data.shape[2])
# ax.set_ylim(data.shape[1], 0)
plt.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
ax.imshow(det_res.data[0], vmin=0.5, vmax=5.0, cmap='gray_r')
for line in det_res.streaks[0].to_numpy():
    ax.plot(line[:4:2], line[1:4:2], c='r', alpha=0.4)
ax.set_xlim(0, det_obj.data.shape[2])
ax.set_ylim(det_obj.data.shape[1], 0)

plt.tight_layout()
plt.show()

# Intensity merging

In [ ]:
if sys.modules.get('dev'): # Maybe sys.modules is better?
    dev = sys.modules.get('dev')
    dev = reload(dev)
else:
    import dev
print(dir(dev))

In [ ]:
setup = cbc.ScanSetup.import_ini('results/exp_geom_232.ini')
table = cbc.CBCTable.import_hdf('results/scan_232_modeled.h5', 'data', setup)
crop = cbc.Crop(roi=(1100, 3260, 1040, 3108))
basis = cbc.Basis.import_ini('results/scan_232_basis.ini')
samples = cbc.ScanSamples.import_dataframe(pd.read_hdf('results/scan_232_samples_trial_4.h5', 'data'))

In [ ]:
part = table.drop_duplicates(method='keep_best')
part = part[part.table['frames'] < 100]
part = part[~part.table['h'].isna()]

In [ ]:
scaler = part.scale((40, 38), basis, samples, 64)

In [ ]:
scaler = scaler.train(1e-3)

In [ ]:
scaler0 = part.scale((40, 38), basis, samples, 64)
scaler0.prof = scaler.prof
scaler0 = scaler0.replace(sfac=np.ones(scaler.signal.size, dtype=np.float32),
                          sfac_err=np.zeros(scaler.signal.size, dtype=np.float32)).update_xtal(1e-3)

In [ ]:
%matplotlib widget
indices = np.argsort(scaler.iidxs[1:] - scaler.iidxs[:-1])
idx = indices[-1]

fig, ax = plt.subplots(1, 2, figsize=(8, 4), gridspec_kw={'width_ratios': [0.1, 1.0]})

slider = Slider(ax=ax[0], label='Streak', valmin=0, valmax=99,
                valstep=np.arange(100), valinit=0, orientation="vertical")

ln0, = ax[1].plot(scaler.signal[scaler.iidxs[idx]:scaler.iidxs[idx + 1]])
ln1, = ax[1].plot(scaler.sfac[scaler.iidxs[idx]:scaler.iidxs[idx + 1]] *
                  scaler.xtal_bi[scaler.iidxs[idx]:scaler.iidxs[idx + 1]] *
                  scaler.prof[scaler.iidxs[idx]:scaler.iidxs[idx + 1]])
ax[1].grid(True)

def update(val):
    idx = indices[-slider.val - 1]
    ln0.set_data(np.arange(scaler.iidxs[idx + 1] - scaler.iidxs[idx]),
                 scaler.signal[scaler.iidxs[idx]:scaler.iidxs[idx + 1]])
    ln1.set_data(np.arange(scaler.iidxs[idx + 1] - scaler.iidxs[idx]),
                 scaler.sfac[scaler.iidxs[idx]:scaler.iidxs[idx + 1]] *
                 scaler.xtal_bi[scaler.iidxs[idx]:scaler.iidxs[idx + 1]] *
                 scaler.prof[scaler.iidxs[idx]:scaler.iidxs[idx + 1]])

    ax[1].relim()
    ax[1].autoscale_view()
    fig.canvas.draw_idle()

slider.on_changed(update)

fig.tight_layout()
plt.show()

In [ ]:
%matplotlib widget
fig, ax = plt.subplots(1, 3, figsize=(8, 4), gridspec_kw={'width_ratios': [0.1, 1.0, 1.0]})

slider = Slider(ax=ax[0], label='Frame', valmin=0, valmax=scaler.xtal.shape[0] - 1,
                valstep=np.arange(0, scaler.xtal.shape[0]),
                valinit=0, orientation="vertical")

im0 = ax[1].imshow(scaler0.xtal.val[0], vmin=0.2, vmax=2.5)
im1 = ax[2].imshow(scaler.xtal.val[0], vmin=0.2, vmax=2.5)

def update(val):
    im0.set_data(scaler0.xtal.val[slider.val])
    im1.set_data(scaler.xtal.val[slider.val])
    fig.canvas.draw_idle()

slider.on_changed(update)

fig.tight_layout()
plt.show()

In [ ]:
scaler.export_sfac('results/test2.hkl', symmetry='4mm')

In [ ]:
sfac = np.empty(scaler.hkl.shape[0])
sfac[scaler.hkl_idxs] = scaler.sfac[scaler.iidxs[:-1]]
sfac_err = np.empty(scaler.hkl.shape[0])
sfac_err[scaler.hkl_idxs] = scaler.sfac_err[scaler.iidxs[:-1]]
norm = sfac_err.mean()
idxs = np.lexsort(np.abs(scaler.hkl.T)[::-1])


hkl_sym = np.abs(scaler.hkl[idxs])
hkl_sym = np.concatenate((np.sort(hkl_sym[:, :2], axis=1), hkl_sym[:, 2:]), axis=1)

hkl_mrg, inv, n_meas = np.unique(hkl_sym, axis=0, return_counts=True,
                                 return_inverse=True)
sfac_mrg = np.zeros(hkl_mrg.shape[0])
dsfac_mrg = np.zeros(hkl_mrg.shape[0])
np.add.at(sfac_mrg, inv, sfac[idxs])
np.add.at(dsfac_mrg, inv, (sfac_err * sfac)[idxs])
dsfac_mrg = dsfac_mrg / sfac_mrg / norm
sfac_mrg = sfac_mrg / n_meas / norm

np.savetxt('results/test2.hkl', np.concatenate((hkl_mrg, sfac_mrg[:, None], dsfac_mrg[:, None],
                                 n_meas[:, None]), axis=1),
           ' %4d %4d %4d %10.2f %10.2f %4d')

In [ ]:
# hkl_file = 'results/test2.hkl'
hkl_file = 'results/1azf.pdb.hkl'
# hkl_file = 'results/scan_232_NI_crystfel.hkl'
hkl_file2 = 'results/test2.hkl'

basis = cbc.Basis.import_ini('results/scan_232_basis.ini')
hkl = np.loadtxt(hkl_file)[:, :3].astype(int)
sfac = np.loadtxt(hkl_file)[:, 3]

rec_vec = hkl.dot(basis.mat)
rec_abs = (rec_vec**2).sum(axis=-1)

hkl2 = np.loadtxt(hkl_file2)[:, :3].astype(int)
sfac2 = np.loadtxt(hkl_file2)[:, 3]

rec_vec2 = hkl2.dot(basis.mat)
rec_abs2 = (rec_vec2**2).sum(axis=-1)

In [ ]:
%matplotlib widget
x = np.linspace(0, 0.10)
sfac_prof = cbc.bin.kr_predict(np.log(sfac) - np.log(sfac.max()),
                           rec_abs[:, None], x[:, None], 1e-3, 4e-3)
sfac_prof2 = cbc.bin.kr_predict(np.log(sfac2) - np.log(sfac2.max()),
                           rec_abs2[:, None], x[:, None], 1e-3, 4e-3)

fig, ax = plt.subplots(figsize=(8, 4))
ax.scatter(rec_abs, np.log(sfac) - np.log(sfac.max()), s=1)
ax.plot(x, sfac_prof, c='r')

ax.scatter(rec_abs2, np.log(sfac2) - np.log(sfac2.max()), s=1)
ax.plot(x, sfac_prof2, c='b')
ax.set_xlim(-0.01, 0.10)
# ax.set_ylim(-10, 0)
plt.show()

# Streak detection, June 2021

In [ ]:
dir_path = '/asap3/petra3/gpfs/p11/2021/data/11010570/raw'
crop = cbc.Crop(roi=(1100, 3260, 1040, 3108))
scan_setup = cbc.ScanSetup.import_ini('results/exp_geom_232.ini')
data_dict = {}

In [ ]:
scan_num = 209 # whitefield, 2s exposure

wf_data = cbc.converter_petra(dir_path, scan_num, transform=crop)
wf_data = wf_data.load('data', processes=16)

wf_data = wf_data.update_mask(method='range-bad', vmax=10000000)
wf_data = wf_data.update_whitefield(method='mean')

In [ ]:
wf_data = wf_data.update_output_file(cbc.CXIStore(f'results/scan_{scan_num:d}_whitefield.h5', 'w'))
wf_data.save(attributes=['mask', 'whitefield', 'data', 'translations', 'tilts'], mode='overwrite')

In [ ]:
scan_num = 209 # whitefield, 2s exposure

file = cbc.CXIStore(f'results/scan_{scan_num:d}_whitefield.h5', 'r')
wf_data = cbc.CrystData(file, transform=None)
wf_data = wf_data.load()

In [ ]:
scan_num = 213 # B12 protein crystal, 1s exposure, 50 frames
pupil = (1050, 1200,  860,  990)

data = cbc.converter_petra(dir_path, scan_num, mask=mask)

In [ ]:
# Lysozyme crystal, 1s exposure, 101 x 4 x 4 frames
# Diffraction starts from frame 505
scan_num = 250
pupil = (1100, 1270,  860, 1070)

h5_dir = os.path.join(dir_path, f'scan_frames/Scan_{scan_num:d}')
h5_files = sorted([os.path.join(h5_dir, path) for path in os.listdir(h5_dir)
                   if path.endswith(('LambdaFar.nxs', '.h5'))])
data_dict[scan_num] = cbc.CrystData(cbc.CXIStore(h5_files, 'r'), crop)

In [ ]:
data_dict[scan_num] = data_dict[scan_num].load(idxs=np.arange(101), processes=8)
data_dict[scan_num] = data_dict[scan_num].update_mask(method='range-bad', vmax=10000000)
data_dict[scan_num] = data_dict[scan_num].update_whitefield('mean')

In [ ]:
output_file = cbc.CXIStore(f'results/scan_{scan_num:d}_whitefield.h5', mode='w')
data_dict[scan_num] = data_dict[scan_num].update_output_file(output_file)
data_dict[scan_num].save(mode='overwrite')

In [ ]:
scan_num = 250

file = cbc.CXIStore(f'results/scan_{scan_num:d}_whitefield.h5')
data_dict[scan_num] = cbc.CrystData(file, transform=None)
data_dict[scan_num] = data_dict[scan_num].load('whitefield')

In [ ]:
# Lysozyme crystal, 2s exposure, 721 frames, full rotation tilt series
scan_num = 232

h5_dir = os.path.join(dir_path, f'scan_frames/Scan_{scan_num:d}')
h5_files = sorted([os.path.join(h5_dir, path) for path in os.listdir(h5_dir)
                   if path.endswith(('LambdaFar.nxs', '.h5'))])
data_dict[scan_num] = cbc.CrystData(cbc.CXIStore(h5_files, 'r'), crop)

In [ ]:
scan_num = 232

data_dict[scan_num] = data_dict[scan_num].clear().load(idxs=np.arange(100), processes=12)
data_dict[scan_num] = data_dict[scan_num].update_mask(method='range-bad', vmax=10000000)
data_dict[scan_num] = data_dict[scan_num].mask_pupil(scan_setup, padding=60)
data_dict[scan_num] = data_dict[scan_num].import_whitefield(data_dict[250].whitefield)
data_dict[scan_num] = data_dict[scan_num].blur_pupil(scan_setup, padding=80, blur=20)

In [ ]:
%matplotlib widget
idx = 4

fig, ax = plt.subplots(1, 2, figsize=(8, 4))
ax[0].imshow(data_dict[scan_num].background[idx], vmax=50)
ax[1].imshow(data_dict[scan_num].cor_data[idx], vmin=0.5, vmax=5)
plt.tight_layout()
plt.show()

In [ ]:
scan_num = 232

det_obj = data_dict[scan_num].lsd_detector()
det_obj = det_obj.generate_pattern(vmin=0.5, vmax=5.0, size=(1, 3, 3))
det_obj = det_obj.update_lsd(quant=2.0e-2, ang_th=45.0)

In [ ]:
%time det_res = det_obj.detect(cutoff=60.0, filter_threshold=0.1, group_threshold=0.5, \
                               dilation=0.5)

In [ ]:
%time det_res = det_res.draw_streaks()

In [ ]:
%time det_res = det_res.draw_background()

In [ ]:
# %time det_res = det_res.update_pattern()

In [ ]:
frame_idx = 80

fig, ax = plt.subplots(figsize=(6, 6))
# ax.imshow(det_obj.pattern[frame_idx], vmin=0.0, vmax=1.0, cmap='gray_r')
ax.imshow(det_res.data[frame_idx], vmin=0.5, vmax=5.0, cmap='gray_r')
for line in det_res.streaks[frame_idx].to_numpy():
    ax.plot(line[:4:2], line[1:4:2], c='r', alpha=0.6)
ax.set_xlim(0, det_obj.data.shape[2])
ax.set_ylim(det_obj.data.shape[1], 0)
plt.tight_layout()
plt.show()

In [ ]:
df = det_res.export_table(concatenate=True)
# df.to_hdf(f'results/scan_232_test.h5', 'data')

In [ ]:
scan_num = 250

file = cbc.CXIStore(f'results/scan_{scan_num:d}_whitefield.h5')
data_dict[scan_num] = cbc.CrystData(file, transform=None)
data_dict[scan_num] = data_dict[scan_num].load('whitefield', processes=2)

In [ ]:
scan_num = 232
data = cbc.converter_petra(dir_path, scan_num, transform=crop,
                           idxs=[])

tables = []
for idxs in tqdm(np.array_split(np.arange(721), 10), total=10, desc=f'Processing scan {scan_num:d}'):
    data = data.clear().load(idxs=idxs, processes=12)
    data = data.update_mask(method='range-bad', vmax=10000000)
    data = data.mask_pupil(scan_setup, padding=60)
    data = data.import_whitefield(data_dict[250].whitefield)
    data = data.blur_pupil(scan_setup, padding=80, blur=20)
    det_obj = data.get_detector()
    det_obj = det_obj.generate_streak_data(vmin=0.5, vmax=5., size=(1, 3, 3))
    det_obj = det_obj.update_lsd(quant=2.0e-2)
    det_res = det_obj.detect(cutoff=70.0, filter_threshold=32.5, group_threshold=0.7)
    det_res = det_res.generate_bgd_mask()
    det_res = det_res.update_streak_data()
    tables.append(det_res.export_table(concatenate=True))

In [ ]:
table = pd.concat(tables)
table.to_hdf('results/scan_232_indexing.h5', 'data')

# Streak detection, May 2022

In [ ]:
dir_path = '/asap3/petra3/gpfs/p11/2022/data/11012881/raw'
crop = cbc.Crop(roi=(800, 3900, 800, 3600))
scan_setup = cbc.ScanSetup.import_ini('results/exp_geom_206.ini')
whitefield = crop.forward(np.load('results/scan_206_whitefield.npy'))
data_dict = {}

In [ ]:
scan_num = 173

data_dict[scan_num] = cbc.converter_petra(dir_path, scan_num,
                                          idxs=[])
data_dict[scan_num] = data_dict[scan_num].load(processes=12)
data_dict[scan_num] = data_dict[scan_num].update_mask(method='range-bad', vmax=10000000)

In [ ]:
# Lysozyme crystal, 0.5s exposure, 18000 frames, full rotation tilt series
scan_num = 207

data_dict[scan_num] = cbc.converter_petra(dir_path, scan_num, transform=crop, idxs=[])

In [ ]:
whitefield = np.load(f'results/scan_{scan_num:d}_whitefield.npy')

In [ ]:
data_dict[scan_num].input_file.indices()

In [ ]:
idxs = np.array_split(data_dict[scan_num].input_file.indices(), 100)[99]

In [ ]:
scan_num = 207

data_dict[scan_num] = data_dict[scan_num].clear().load(idxs=idxs, processes=16)
data_dict[scan_num] = data_dict[scan_num].update_mask(method='range-bad', vmax=40)
data_dict[scan_num] = data_dict[scan_num].mask_pupil(scan_setup, padding=60)
data_dict[scan_num] = data_dict[scan_num].import_whitefield(data_dict[scan_num].transform.forward(whitefield))
data_dict[scan_num] = data_dict[scan_num].blur_pupil(scan_setup, padding=80, blur=20)

In [ ]:
%matplotlib widget
idx = 104

fig, ax = plt.subplots(1, 1, figsize=(8, 8))
ax.imshow(data_dict[scan_num].cor_data[idx], vmax=3)
plt.tight_layout()
plt.show()

In [ ]:
det_obj = data_dict[scan_num].get_detector()
det_obj = det_obj.generate_streak_data(vmin=0.3, vmax=2.0, size=(1, 3, 3))
det_obj = det_obj.update_lsd(quant=1.8e-2)

In [ ]:
%time det_res = det_obj.detect(cutoff=70.0, filter_threshold=6.0, group_threshold=0.4)

In [ ]:
%time det_res = det_res.generate_bgd_mask()

In [ ]:
%time det_res = det_res.update_streak_data()

In [ ]:
frame_idx = 0

fig, ax = plt.subplots(figsize=(6, 6))
# ax.imshow(det_res.data[frame_idx], vmin=0.2, vmax=2.0, cmap='gray_r')
ax.imshow(det_obj.streak_data[frame_idx], vmax=1.0, cmap='gray_r')
# for line in det_res.streaks[det_res.frames[frame_idx]].to_numpy():
#     ax.plot(line[:4:2], line[1:4:2], c='r', alpha=0.4)
ax.set_xlim(0, det_obj.data.shape[2])
ax.set_ylim(det_obj.data.shape[1], 0)
plt.tight_layout()
plt.show()

In [ ]:
scan_setup = cbc.ScanSetup.import_ini('results/exp_geom_206.ini')
streaks = cbc.ScanStreaks.import_hdf('results/scan_206_indexing.h5', 'data', scan_setup)
data = cbc.CrystData(cbc.CXIStore('results/scan_206_data.h5'))

In [ ]:
frame_idx = 460

pattern = streaks.pattern_image(frame_idx)
data = data.clear().load(idxs=np.array([160]))

In [ ]:
det_obj = data.get_detector()
det_obj = det_obj.generate_streak_data(vmin=0.3, vmax=2.0, size=(1, 3, 3))
det_obj = det_obj.update_lsd(quant=1.8e-2)

In [ ]:
%time det_res = det_obj.detect(cutoff=70.0, filter_threshold=6.0, group_threshold=0.4)
%time det_res = det_res.generate_bgd_mask()
%time det_res = det_res.update_streak_data()

In [ ]:
%matplotlib widget
fig, axes = plt.subplots(1, 2, figsize=(8, 4))
axes[0].imshow(pattern, vmax=0.3, cmap='gray_r')
axes[1].imshow(data['cor_data'][0], vmax=2.0, cmap='gray_r')
for line in det_res.streaks[frame_idx].to_numpy():
    axes[1].plot(line[:4:2], line[1:4:2], c='r', alpha=0.4)
plt.tight_layout()
plt.show()

# CBC indexing

In [ ]:
scan_setup = cbc.ScanSetup.import_ini('results/exp_geom_232.ini')
table = cbc.CBCTable.import_hdf('results/scan_232_detected.h5', 'data', scan_setup)
crop = cbc.Crop(roi=(1100, 3260, 1040, 3108))
basis = cbc.Basis.import_ini('results/scan_232_basis.ini')
samples = cbc.ScanSamples.import_dataframe(pd.read_hdf('results/scan_232_samples_trial_4.h5', 'data'))

In [ ]:
dataframes = []
for frame, sample in tqdm(samples.items(), total=len(samples)):
    problem = table.refine_sample(frame, (0.0, 0.0, 0.0), basis, sample, 0.5, 4.0)
    dataframes.append(problem.index_frame(problem.x0, frame, num_threads=64))

In [ ]:
table_idx = pd.concat(dataframes)
table_idx.to_hdf('results/scan_232_indexed.h5', 'data')

In [ ]:
%matplotlib widget
fig, ax = plt.subplots(figsize=(8, 8))
ax.imshow(table.pattern_image(62))
plt.show()

In [ ]:
qsize = (512, 512, 512)
q_bounds = (0.25, 0.25, 0.25)

qx_arr = np.linspace(-q_bounds[0], q_bounds[0], qsize[0], endpoint=True)
qy_arr = np.linspace(-q_bounds[1], q_bounds[1], qsize[1], endpoint=True)
qz_arr = np.linspace(-q_bounds[2], q_bounds[2], qsize[2], endpoint=True)
qmap = table.create_qmap(samples, qx_arr, qy_arr, qz_arr)

In [ ]:
%matplotlib widget
fig, ax = plt.subplots(figsize=(6, 6))
ax.imshow(qmap.val[:, qmap.shape[1] // 2 - 5:qmap.shape[1] // 2 + 5].mean(axis=1), vmax=10.0, cmap='gray_r')
plt.tight_layout()
plt.show()

In [ ]:
rmap = qmap.fft(num_threads=64)

In [ ]:
if sys.modules.get('dev'): # Maybe sys.modules is better?
    dev = sys.modules.get('dev')
    dev = reload(dev)
else:
    import dev
print(dir(dev))

In [ ]:
%time wdw = dev.filter_direction(rmap.grid, [0.0, 1.0, 0.0], 5.0, 1.0, num_threads=64)

In [ ]:
%time rmap = qmap.fft(num_threads=64)
%time rmap = rmap.gaussian_blur(1.0, num_threads=64)

In [ ]:
%time rmap = rmap.white_tophat(np.ones((9, 9, 9)), num_threads=64).clip(0.0, np.inf)

In [ ]:
%time rmap.val /= np.max(rmap.val)

In [ ]:
peaks = rmap.find_peaks(0.1)
basis = rmap.find_peaks(0.1, reduce=True)

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter3d(x=peaks[:, 1], y=peaks[:, 0], z=peaks[:, 2], mode='markers',
                           marker=dict(size=6)))
fig.add_trace(go.Scatter3d(x=basis[:, 1], y=basis[:, 0], z=basis[:, 2], mode='markers',
                           marker=dict(size=8)))

fig.update_layout(width=800, height=800, margin=dict(l=20, r=20, b=20, t=20))
fig.show()

In [ ]:
rfit = rmap[::8, ::8, ::8]

In [ ]:
x0 = basis.ravel()
bounds = np.sort(np.stack((0.9 * x0, 1.1 * x0), axis=1), axis=1)
res = minimize(rfit.criterion, x0, jac=True, method='L-BFGS-B', bounds=bounds,
               args=(100.0, 5.0, 1e-12, 64))

In [ ]:
# Lattice basis in meters
basis * scan_setup.wavelength

In [ ]:
fig = go.Figure()

fig.add_trace(go.Volume(x=rx_fit.flatten(),
                        y=ry_fit.flatten(),
                        z=rz_fit.flatten(),
                        value=r_fit.flatten(),
                        isomin=1e-5, isomax=1e-3,
                        opacity=0.1, # needs to be small to see through all surfaces
                        surface_count=20))
# fig.add_trace(go.Scatter3d(x=axes[:, 1], y=axes[:, 0], z=axes[:, 2], mode='markers'))
fig.add_trace(go.Scatter3d(x=res.x[1::3], y=res.x[::3], z=res.x[2::3], mode='markers'))

fig.update_layout(width=800, height=800, margin=dict(l=20, r=20, b=20, t=20))
fig.show()

# CBC Model

In [ ]:
scan_num = 232

dir_path = '/asap3/petra3/gpfs/p11/2021/data/11010570/raw'
log_path = os.path.join(dir_path, f'server_log/Scan_logs/Scan_{scan_num:d}.log')
log_data = cbc.LogContainer().read_logs(log_path).read_translations()

In [ ]:
scan_setup = cbc.ScanSetup.import_ini('results/exp_geom_232.ini')
smp_pos = np.array([0.14343635, 0.16275142, 0.38192596])
samples = log_data.generate_samples(smp_pos, scan_setup)
# samples.to_dataframe().to_hdf('results/scan_232_samples_trial_3.h5', 'data')

In [ ]:
scan_setup = cbc.ScanSetup.import_ini('results/exp_geom_232.ini')
table = cbc.CBCTable.import_hdf('results/scan_232_detected.h5', 'data', scan_setup)
crop = cbc.Crop(roi=(1100, 3260, 1040, 3108))
basis = cbc.Basis.import_ini('results/scan_232_basis.ini')
samples = cbc.ScanSamples.import_dataframe(pd.read_hdf('results/scan_232_samples_trial_4.h5', 'data'))

In [ ]:
frame_idx = 0

problem = table.refine_sample(frame_idx, (0.0, 0.0, 0.0), basis, samples[frame_idx], 0.3, 4.0, 0.0)

In [ ]:
samples_reg = samples.regularise((5e0, 5e1))

In [ ]:
scan_num = 232
dir_path = '/asap3/petra3/gpfs/p11/2021/data/11010570/raw'
h5_dir = os.path.join(dir_path, f'scan_frames/Scan_{scan_num:d}')
h5_files = sorted([os.path.join(h5_dir, path) for path in os.listdir(h5_dir)
                   if path.endswith(('LambdaFar.nxs', '.h5'))])
data = cbc.CrystData(cbc.CXIStore(h5_files), transform=crop)

wf_data = cbc.CrystData(cbc.CXIStore(f'results/scan_250_whitefield.h5')).load('whitefield')

In [ ]:
tilts = np.array([sample.rotation.to_tilt() for sample in samples.values()])

In [ ]:
%matplotlib widget
fig, ax = plt.subplots(figsize=(8, 4))
ax.plot(tilts[:, 0])
ax.plot(tilts[:, 1])
ax.plot(tilts[:, 2])
plt.show()

In [ ]:
%matplotlib widget
fig, ax = plt.subplots(figsize=(8, 4))
ax.plot(samples.get_positions(1))
ax.plot(samples_reg.get_positions(1))
ax.grid(True)
# ax.plot(samples.get_positions(1) / samples.get_positions(1)[0])
# ax.plot(samples.get_positions(2) / samples.get_positions(2)[0])

plt.show()

In [ ]:
problem = table.refine_sample(frame_idx, (2e-2, 2e-3, 0.0), basis, samples[frame_idx],
                              0.3, 3.0, 0.5)

In [ ]:
def criterion(x, problem):
    return problem.fitness(x)[0]

res = differential_evolution(criterion, bounds=np.stack(problem.get_bounds()).T,
                             maxiter=100, popsize=25, workers=64, updating='deferred',
                             args=(problem,))

In [ ]:
uda = pygmo.de(gen=100)
algo = pygmo.algorithm(uda)
prob = pygmo.problem(problem)
pops = [pygmo.population(size=30, prob=prob, b=pygmo.bfe()) for _ in range(32)]
archi = pygmo.archipelago()
for pop in pops:
    archi.push_back(algo=algo, pop=pop)

In [ ]:
archi.evolve()
%time archi.wait()
x = archi.get_champions_x()[np.argmin(archi.get_champions_f())]

# CBC Geometry calibration

In [ ]:
# Experimental geometry

log_prt = cbc.LogProtocol.import_default()

foc_z = -0.012028122 # From PXST
dir_path = '/asap3/petra3/gpfs/p11/2021/data/11010570/raw'
h5_dir = os.path.join(dir_path, 'scan_frames/Scan_253')
log_path = os.path.join(dir_path, 'server_log/Scan_logs/Scan_253.log')
h5_files = sorted([os.path.join(h5_dir, path) for path in os.listdir(h5_dir)
                   if path.endswith(('LambdaFar.nxs', '.h5')) and not path.endswith('master.h5')])
h5_master = [os.path.join(h5_dir, path) for path in os.listdir(h5_dir)
             if path.endswith('master.h5')][0]

defocus = log_prt.load_attributes(log_path)['Session logged attributes']['z_sample'] - foc_z
file = cbc.CXIStore(h5_master)
with file:
    x_pixel_size = file.load_attribute('x_pixel_size').item()
    y_pixel_size = file.load_attribute('y_pixel_size').item()
wl = 2.0664032162696132e-11 # 60 keV

In [ ]:
pupil = np.array([2210, 2345, 1957, 2090])
pupil_2 = np.array([2214, 2400, 1974, 2155])
db = np.array([2171, 1913])
db_2 = np.array([2159, 1913])

In [ ]:
det_dist_2 = 0.15 / (1.0 - 0.718) # sample-to-detector distance for scan 254
det_dist = det_dist_2 - 0.15 # sample-to-detector distance for scan 253

In [ ]:
center = db + np.mean(((pupil - np.repeat(db, 2)) * defocus / det_dist).reshape((2, 2)), axis=1)
center_2 = db_2 + np.mean(((pupil_2 - np.repeat(db_2, 2)) * defocus / det_dist_2).reshape((2, 2)), axis=1)
tilt_shift = np.arange(4362)**2 / 2159**2 * 12.0 # Y_253 = Y_254 + tilt_shift[Y_254]
center[0], center_2[0] + tilt_shift[int(center_2[0])]

In [ ]:
delta_x = (pupil[2] - db[1]) * x_pixel_size
delta_y = (pupil[0] - db[0]) * y_pixel_size
phis = np.arctan2(delta_y, delta_x)
thetas = np.arctan(np.sqrt(delta_x**2 + delta_y**2) / det_dist)
kin_min = np.stack((np.sin(thetas) * np.cos(phis), np.sin(thetas) * np.sin(phis), np.cos(thetas)))

In [ ]:
delta_x = (pupil[3] - db[1]) * x_pixel_size
delta_y = (pupil[1] - db[0]) * y_pixel_size
phis = np.arctan2(delta_y, delta_x)
thetas = np.arctan(np.sqrt(delta_x**2 + delta_y**2) / det_dist)
kin_max = np.stack((np.sin(thetas) * np.cos(phis), np.sin(thetas) * np.sin(phis), np.cos(thetas)))

In [ ]:
smp_pos = np.array([center[1] * x_pixel_size,
                    center[0] * y_pixel_size, det_dist])
foc_pos = np.array([db[1] * x_pixel_size, db[0] * y_pixel_size, det_dist + defocus])
scan_setup = cbc.ScanSetup(smp_pos=smp_pos, foc_pos=foc_pos, rot_axis=np.array([0.0, 1.0, 0.0]),
                           kin_min=kin_min, kin_max=kin_max, wavelength=wl, x_pixel_size=x_pixel_size,
                           y_pixel_size=y_pixel_size)

In [ ]:
with open('results/exp_geom_248_253.ini', 'w') as ini_file:
    scan_setup.export_ini().write(ini_file)

In [ ]:
smp_pos_2 = np.array([center_2[1] * x_pixel_size,
                      center_2[0] * y_pixel_size, det_dist_2])
foc_pos_2 = np.array([db_2[1] * x_pixel_size, db_2[0] * y_pixel_size, det_dist_2 + defocus])
scan_setup_2 = cbc.ScanSetup(smp_pos=smp_pos_2, foc_pos=foc_pos_2, rot_axis=np.array([0.0, 1.0, 0.0]),
                             wavelength=wl, x_pixel_size=x_pixel_size, y_pixel_size=y_pixel_size)

In [ ]:
with h5py.File('results/scan_250_proc.h5', 'r') as cxi_file:
    whitefield = cxi_loader.read_cxi('whitefield', cxi_file)
    mask = cxi_loader.read_cxi('mask', cxi_file)[0]

In [ ]:
scan_num = 224 # Det dist = 20 cm
data = cbc.converter_petra(dir_path, scan_num, mask=mask,
                           pupil=(2200, 2370, 1900, 2110),
                           roi=(1100, 3260, 1040, 3108),
                           whitefield=whitefield)

In [ ]:
scan_num = 225 # Det dist = 40 cm
data = cbc.converter_petra(dir_path, scan_num, mask=mask,
                           pupil=(2200, 2370, 1900, 2110),
                           roi=(1100, 3260, 1040, 3108),
                           whitefield=whitefield)

In [ ]:
scan_num = 253 # Det dist = 40 cm

data = cbc.converter_petra(dir_path, scan_num,
                           pupil=(2200, 2370, 1900, 2110),
                           roi=(1100, 3260, 1040, 3108))
data.data = np.sum(data.data, axis=0, keepdims=True)
data.good_frames = data.good_frames[[0]]
data = data.import_mask(mask, update='multiply')
data = data.import_whitefield(3.0 * whitefield)

In [ ]:
det_obj = cbc.StreakDetector.import_data(data, 0.2, 3., (1, 3, 3))
det_obj = det_obj.update_lsd(scale=0.9, sigma_scale=0.9, log_eps=0, ang_th=60, density_th=0.5, quant=1.5e-2)
det_obj = det_obj.update_mask(dilation=8)
det_obj = det_obj.update_streak_data(iterations=4)

In [ ]:
scan_num = 254 # Det dist = 55 cm
whitefield_2 = np.zeros(whitefield.shape, dtype=np.float64)
whitefield_2[29:, 41:] = whitefield[:-29, :-41]
mask_2 = np.ones(mask.shape, dtype=bool)
mask_2[29:, 41:] = mask[:-29, :-41]

data_2 = cbc.converter_petra(dir_path, scan_num,
                             pupil=(2140, 2450, 1900, 2200),
                             roi=(700, 3660, 640, 3508))

data_2.data = np.sum(data_2.data, axis=0, keepdims=True)
data_2.good_frames = data_2.good_frames[[0]]
data_2 = data_2.import_mask(mask * mask_2, update='multiply')
data_2 = data_2.import_whitefield(3.0 * whitefield)

In [ ]:
det_obj_2 = cbc.StreakDetector.import_data(data_2, 0.2, 3., (1, 3, 3))
det_obj_2 = det_obj_2.update_lsd(quant=2.8e-2)
det_obj_2 = det_obj_2.update_mask(dilation=8)
det_obj_2 = det_obj_2.update_streak_data(iterations=4)

In [ ]:
%matplotlib notebook
frame_idx = 0
scale = 0.718


fig, ax = plt.subplots(1, 1, figsize=(7, 7))
ax.imshow(data.get('cor_data')[frame_idx], vmin=0.2, vmax=3., cmap='gray_r')
for line in det_obj.streaks[frame_idx]:
    ln = ax.plot(line[:4:2], line[1:4:2], line[4], c='r', alpha=0.6)[0]
ln.set_label('Scan 253')
for line in det_obj_2.streaks[frame_idx]:
    ln = ax.plot((line[:4:2] + data_2.roi[2] - center[1]) * scale + center[1] - data.roi[2],
                 (line[1:4:2] + data_2.roi[0] - center[0]) * scale + center[0] - data.roi[0]
                 + tilt_shift[line[1:4:2].astype(int)], line[4], c='g', alpha=0.6)[0]
ln.set_label('Scan 254, with a tilt')
for line in det_obj_2.streaks[frame_idx]:
    ln = ax.plot((line[:4:2] + data_2.roi[2] - center[1]) * scale + center[1] - data.roi[2],
                 (line[1:4:2] + data_2.roi[0] - center[0]) * scale + center[0] - data.roi[0],
                 line[4], c='b', alpha=0.3)[0]
ln.set_label('Scan 254, no tilt')
ax.scatter(1913 - data.roi[2], 2159 + tilt_shift[2159] - data.roi[0], s=50, c='r')
ax.set_xlim(0, data.roi[3] - data.roi[2])
ax.set_ylim(data.roi[1] - data.roi[0], 0)
ax.legend(fontsize=15)
plt.tight_layout()
plt.savefig('figures/scan_253_254.jpg', dpi=300)

In [ ]:
%matplotlib notebook

fig, ax = plt.subplots(1, 2, figsize=(9, 5))
ax[0].imshow(data.data[2, 2140:2450, 1900:2200], vmax=500)
ax[1].imshow(data_2.data[2, 2140:2450, 1900:2200], vmax=500)
plt.show()

smp_pos = [0.15 / (1.0 - 0.718)

In [ ]:
%matplotlib notebook

fig, ax = plt.subplots(1, 2, figsize=(10, 5))
ax[0].imshow(data.data[0, 2210:2344, 1957:2090], vmax=300000)
ax[1].imshow(data_2.data[0, 2214:2400, 1974:2155], vmax=150000)
plt.show()